Task Title: Sentiment Analysis using DistilBERT for Twitter Financial News
 Task Description: In this code snippet, you will perform sentiment analysis on financial
news-related tweets using the DistilBERT model. The goal is to classify tweets into one of
three sentiment labels: "Bearish", "Bullish", or "Neutral". The dataset can be downloaded with
code below and consists of annotated finance-related tweets with sentiment labels. The
dataset is divided into training and validation subsets. The code will tokenize the tweet texts,
train a DistilBERT-based sentiment classifier, and evaluate its performance.
 You must use HF Trainer or Lightning Trainer for Training/Inference Loops

In [ ]:
import datasets
import torch
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

device = "cuda" if torch.cuda.is_available() else "cpu"


#loading/dowloading dataset

dataset = load_dataset("zeroshot/twitter-financial-news-sentiment", cache_dir="../CentralCache")
model =AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3,cache_dir="../CentralCache")
#Tokenzing the text in dataset

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", cache_dir="../CentralCache")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
tokenized_datasets = tokenized_datasets.remove_columns(["text"])  # Keep input IDs and labels only
tokenized_datasets.set_format("torch")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

sent_train.csv:   0%|          | 0.00/859k [00:00<?, ?B/s]

sent_valid.csv:   0%|          | 0.00/217k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9543 [00:00<?, ? examples/s]

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/9543 [00:00<?, ? examples/s]

Map:   0%|          | 0/2388 [00:00<?, ? examples/s]

In [ ]:
#Training Setup

from transformers import TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    learning_rate=0.01,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    eval_steps=1000,
    save_steps=1000,
    weight_decay=0.01,
    # Remove save_strategy="epoch"
    save_strategy="steps",  # Changed to 'epoch' to match evaluation_strategy
    logging_dir="./logs",
    logging_steps=1000,
    load_best_model_at_end=True,
    max_steps=10 # Stop after 3 steps
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


<ipython-input-8-375a1d6f084b>:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()
# Evaluate and print metrics
results = trainer.evaluate()
#print(results)
# Save the model, tokenizer, and training arguments
drive_path = "/content/drive/MyDrive/sentiment_model"
model.save_pretrained(drive_path)
tokenizer.save_pretrained(drive_path)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


('/content/drive/MyDrive/sentiment_model/tokenizer_config.json',
 '/content/drive/MyDrive/sentiment_model/special_tokens_map.json',
 '/content/drive/MyDrive/sentiment_model/vocab.txt',
 '/content/drive/MyDrive/sentiment_model/added_tokens.json',
 '/content/drive/MyDrive/sentiment_model/tokenizer.json')

In [ ]:
def predict_sentiment(texts):
    tokens = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=128)
    outputs = model(**tokens)
    predictions = outputs.logits.argmax(dim=1).numpy()
    sentiment_map = {0: "Bearish", 1: "Neutral", 2: "Bullish"}
    return [sentiment_map[pred] for pred in predictions]

# Example tweets
sample_tweets = ["The market looks strong today.", "I'm worried about the economy.", "No clear trend is visible."]
print(predict_sentiment(sample_tweets))


['Bullish', 'Bullish', 'Bullish']
